# Automated ML

In [76]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview

In [77]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Udacity-exp-p3'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [78]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Telecom Churn"
description_text = "Telecom Churn"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/nandex7/Capstone-Project---Azure-Machine-Learning-Engineer/main/data/WA_Fn-UseC_-Telco-Customer-Churn.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


In [79]:
dataset.take(5).to_pandas_dataframe()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,True,False,1,False,No phone service,DSL,No,...,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,5575-GNVDE,Male,0,False,False,34,True,No,DSL,Yes,...,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,3668-QPYBK,Male,0,False,False,2,True,No,DSL,Yes,...,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,7795-CFOCW,Male,0,False,False,45,False,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,9237-HQITU,Female,0,False,False,2,True,No,Fiber optic,No,...,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True


In [80]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-automl"
# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)
# For a more detailed view of current AmlCompute status, use get_status().

Creating
Succeeded..............................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

This creates a general AutoML settings object. These inputs must match what was used when training in the portal. label_column_name has to be y for example.

In [81]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric": 'accuracy'
}


automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Churn",   
                             #path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [82]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5,
    "featurization": 'auto'
}
# TODO: Put your automl config here
automl_config = AutoMLConfig(task = "classification",
                             training_data=dataset,
                             compute_target=compute_target,
                             label_column_name="Churn",   
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [83]:
#from azureml.automl.core.featurization import FeaturizationConfig
#featurization_config = FeaturizationConfig()
# Drop columns before training
#featurization_config.drop_columns = ['customerID']

In [84]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [85]:

from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)
assert(remote_run.get_status() == 'Completed')

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



ERROR: Setup iteration failed: Unidentified error, check logs in portal / compute


AssertionError: 

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()
best_metrics = best_run.get_metrics()
print(best_run)

In [ ]:
print(fitted_model)

In [ ]:
print(best_metrics)

In [73]:
import joblib 

#best_run, fit_model = remote_run.get_output()

joblib.dump(value=fitted_model, filename="BestAutoMLmodel.pkl")

['BestAutoMLmodel.pkl']

In [ ]:
import pickle
file = open("./BestAutoMLmodel.pkl",'rb')
aml_model = pickle.load(file)
file.close()

##filename = 'best_automl_model.sav'
##  joblib.dump(fitted,filename)

In [ ]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://raw.githubusercontent.com/nandex7/Capstone-Project---Azure-Machine-Learning-Engineer/main/data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['Churn'])]

y_test = df_test['Churn']
X_test = df_test.drop(['Churn'], axis=1)

In [ ]:
print(accuracy_score(aml_model.predict(X_test, y_test))

In [ ]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [ ]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
script = './score.py'
remote_run.download_file('outputs/scoring_file_v_1_0_0.py', './score.py')
print(remote_run.model_id)


In [ ]:
model = remote_run.register_model(model_name= 'automlbestmodel')

In [ ]:
from azureml.core import Model
from azureml.core.webservice import AciWebservice

env = automl_best_run.get_environment()    

inference_config = InferenceConfig(entry_script=script,
                                   environment=env)

deployment_config =AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1,
                                               enable_app_insights=True,
                                               auth_enabled=True,
                                              )

aservice = Model.deploy(ws, "Telcom Model Deploy", [model], inference_config, deployment_config)
aservice.wait_for_deployment(show_output = True)
print(aservice.state)

print(aservice.scoring_uri)

pri_key, sec_key = aservice.get_keys()
print(pri_key)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
print(aservice.scoring_uri)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
import requests
scoring_uri='http://212f2021-c0d2-494a-b284-426d0a8b6722.southcentralus.azurecontainer.io/score'
key=''
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
data = {"data":
        [
          {
            "gender": 68,
            "SeniorCitizen": 1,
            "Partner":Yes ,
            "Dependents": No,
            "tenure": 1,
            "PhoneService": No,
            "MultipleLines": No,
            "InternetService": DSL,
            "OnlineSecurity": No,
            "OnlineBackup": Yes,
            "DeviceProtection": No,
            "TechSupport": No,
            "StreamingTV": No,
            "StreamingMovies": No,
            "Contract":Month-to-month,
            "PaperlessBilling":Yes,
            "PaymentMethod":Electronic check,
            "MonthlyCharges":29.85,
            "TotalCharges":29.85
            
          }
      ]
    }
test_sample = json.dumps(data)


resp = requests.post(scoring_uri, test_sample, headers=headers)
print(resp.json())

In [ ]:
aservice.delete()
print("Service deleted")

In [ ]:
compute_target.delete()
print("Compute deleted")